In [1]:
import pandas
import numpy as np
import random
import math

In [2]:
test=pandas.read_csv("../data/df_test.csv")
train=pandas.read_csv("../data/df_train.csv")

In [3]:
def matrix_data(tt): 
    tt_nl=tt.loc[:, tt.columns != "type"]
    tt_l=tt.loc[:, tt.columns == "type"]
    thing=[[adiff(float(j)) for j in i] for i in tt_nl.values]
    return(thing,tt_l)

In [4]:
train.shape

(149, 10)

In [5]:
from collections import Counter

In [6]:
print(Counter(test["type"]))

Counter({2: 23, 1: 21, 7: 9, 3: 5, 5: 4, 6: 3})


In [7]:
##normalise data before 

In [8]:
def next_layer(previous_layer,relevant_weights):
    bias=0.5
    train_nl,train_l=matrix_data(train)
    weights=np.random.rand(train_nl.shape[1],10)
    activation(train_nl@weights+bias)

In [27]:
import traceback
class adiff:
    
    def __init__(self, value, parents=None):
        try:
            assert type(value) in {float, int}, "Numbers please"
        except AssertionError:
            print(traceback.format_exc()) 
            print(type(value))           
        self.val = value
        self.gradient = 0.0
        self.parents = parents
        if parents == None:
            self.parents = []

    """The backprop, from Rasmusbergpalm's github"""
    # def backprop(self, bp):
    #     self.grad += bp
    #     for parent, grad in self.parents:
    #         parent.backprop(grad * bp)
    #     return self
    def backprop(self, value):
        self.gradient+= value
        for parent, gradient in self.parents:
            parent.backprop(gradient * value)
    
    def backward(self):
        self.backprop(1.0)

    def __add__(self, o_value):
        n_value = self.val + o_value.val
        return adiff(n_value, [(self, 1.0), (o_value, 1.0)])

    def __mul__(self, o_value):
        n_value = self.val * o_value.val
        return adiff(n_value, [(self, o_value.val), (o_value, self.val)])

    def __sub__(self, o_value):
        n_value = adiff(self.val - o_value.val, [(self, 1.0), (o_value, 1.0)])
        return n_value
        
    def __pow__(self, o_value):
        n_value = self.val**o_value.val
        return adiff(n_value, [(self, o_value.val*self.val**(o_value.val-1.0)), (o_value, n_value*math.log(self.val))])
    
    def __truediv__(self, o_value):
        n_value = self.val/o_value.val
        return adiff(n_value,[(self,1.0/o_value.val), (o_value,-self.val/o_value.val**2)])##!!!!!!

    #Returning a f-string, with the value and gradient
    def __repr__(self):
        return (f"adiff value = {self.val}, parents = {len(self.parents)}, gradient = {self.gradient}")

    def e(self):
        n_value=float(np.exp(self.val))
        return adiff(n_value, [(self, n_value)])
    
    def relu(self,data):
        return [[adiff(j.val,[(j,1.0)]) if j.val>0 else adiff(0,[(j,0.0)]) for j in i] for i in data]    

    def softmax(self, data):
        sums=[]
        for i in range(len(data[0])):
            column = [row[i] for row in data]
            current_sum = adiff(0.0)
            for k in column:
                current_sum += k.e()
            sums.append(current_sum)
        return [[(data[ri][ci]).e()/sums[ci] for ci in range(len(data[0]))]for ri in range(len(data))]

    def sigmoid(self, data):
        print("in sigmoid",data)
        thing=[]
        for i in data:
            print("i",i)
            thing2=[]
            for j in i:
                print("heyo")
                print(j.e())
                print(adiff(1.0)/(adiff(1.0)+j.e()))
                print("this wont print")
                thing2.append(adiff(1.0)/(adiff(1.0)+j.e()))
            thing.append(thing2)
        return thing2#[[adiff(1.0)/(adiff(1.0)+j.e()) for j in i] for i in data] 
## 1.0 / (1.0 + np.exp(-data))
##sigmoid 


In [18]:
class network:
    def __init__(self, layers_info,activate = "sigmoid"):
        #self.n_features = n_features
        self.bias = 0.5
        self.layers_info = layers_info
        self.Network=[]
        self.activate=activate
    
    def forward_pass(self,input_data):
        #create new layers here 
        for l_info in self.layers_info:
            input_data = np.array(input_data)
            new_layer = layer(l_info, len(input_data[1]),self.activate)
            new_layer.calculation(input_data)
            self.Network.append(new_layer)
            input_data = new_layer.next_layer_input
        output_layer= layer(6, len(input_data[1]),"softmax")
        output_layer.calculation(input_data)
        self.Network.append(output_layer)
    
    def mse_loss(y_true, y_pred):
        return ((y_true-y_pred)**2).mean()
        
    def get_layer(self, i):
        return self.network[i]
    

class layer:
    def __init__(self,number_of_neurons, n_prev_neurons,activate):
        self.n = number_of_neurons
        self.weights = [[adiff(random.random()) for i in range(number_of_neurons)] for _ in range(n_prev_neurons)]
        self.bias = [adiff(0.5) for i in range (number_of_neurons)]#this should be a parameter can be in the weight matrix mus be updated alongside the weights 
        self.next_layer_input = None
        self.activate=activate
    
    def activation(self,data,activate):
        if activate == "ReLU":
            return adiff.relu(self,data)##is self needed here check later
        elif activate == "sigmoid":
            return  adiff.sigmoid(self,data)
        elif activate == "softmax":
            return adiff.softmax(self,data)
            return np.exp(data)/(np.exp(data).sum(axis=1).reshape(-1,1))
        else:
            print("invalid activaiton function name")
        #more here if needed
        #choose activation function here

    def dot_llist(self,llist1,llist2):
        dot_product = [[0.0 for i in range(len(llist2[0]))] for j in range(len(llist1))]
        for row in range(len(llist1)):
            for column in range(len(llist2[0])):
                a = llist1[row]
                b = [l2row[column] for l2row in llist2]
                sum_ = adiff(0.0)
                for f in a:
                    for g in b:
                        sum_ += f * g
                dot_product[row][column]=sum_
        return dot_product

    def add_bias(self,dw):
        return [[dw[rowi][columni]+self.bias[columni] for columni in range(len(dw[0]))]for rowi in range(len(dw))]

    
    def calculation(self, data):
        print(self.add_bias(self.dot_llist(data,self.weights)))
        self.next_layer_input = self.activation(self.add_bias(self.dot_llist(data,self.weights)), self.activate)##make list of list friendly

In [11]:
np.array([[1,1,1],[2,2,2],[3,3,3]])+np.array([[1,10,100]])[None,:]## dividing by a row vector thing dont delete yet

array([[[  2,  11, 101],
        [  3,  12, 102],
        [  4,  13, 103]]])

In [51]:
tt_nl=train.loc[:, train.columns != "type"]
x = norm.fit_transform(tt_nl)#do this before making adiff later
x=[[adiff(float(j)) for j in i] for i in x]

In [53]:
from sklearn.preprocessing import StandardScaler
norm = StandardScaler()
#x=matrix_data(train)[0]
testingthing=network([10,3,4,7],"sigmoid")
testingthing.forward_pass(x)


adiff value = 0.004117005140104657, parents = 1, gradient = 0.0
adiff value = 0.9958998750952034, parents = 2, gradient = 0.0
this wont print
heyo
adiff value = 0.03968785734543585, parents = 1, gradient = 0.0
adiff value = 0.9618271416127067, parents = 2, gradient = 0.0
this wont print
heyo
adiff value = 0.02789764442684323, parents = 1, gradient = 0.0
adiff value = 0.9728595112771184, parents = 2, gradient = 0.0
this wont print
heyo
adiff value = 0.004928542592066734, parents = 1, gradient = 0.0
adiff value = 0.9950956288102294, parents = 2, gradient = 0.0
this wont print
i [adiff value = 7.566920888194813, parents = 2, gradient = 0.0, adiff value = 4.9504798904824785, parents = 2, gradient = 0.0, adiff value = 6.657063233699082, parents = 2, gradient = 0.0, adiff value = 8.488038989620145, parents = 2, gradient = 0.0, adiff value = 7.184402478636161, parents = 2, gradient = 0.0, adiff value = 6.4587072431248505, parents = 2, gradient = 0.0, adiff value = 7.578103069561059, parents =

TypeError: object of type 'adiff' has no len()

In [ ]:
testingthing.Network[4].next_layer_input[0]

[adiff value = nan, parents = 2, gradient = 0.0,
 adiff value = nan, parents = 2, gradient = 0.0,
 adiff value = nan, parents = 2, gradient = 0.0,
 adiff value = nan, parents = 2, gradient = 0.0,
 adiff value = nan, parents = 2, gradient = 0.0,
 adiff value = nan, parents = 2, gradient = 0.0]

In [ ]:
adiff(10000000000000000000000000000000000000000000000000000000000000000.0)/adiff(0.0)

ZeroDivisionError: float division by zero

In [ ]:
data=matrix_data(train)[0]
data
#data=[[j if j.val>0 else adiff(0) for j in i] for i in data]


[[adiff value = 1.51839, parents = 0, gradient = 0.0,
  adiff value = 12.85, parents = 0, gradient = 0.0,
  adiff value = 3.67, parents = 0, gradient = 0.0,
  adiff value = 1.24, parents = 0, gradient = 0.0,
  adiff value = 72.57, parents = 0, gradient = 0.0,
  adiff value = 0.62, parents = 0, gradient = 0.0,
  adiff value = 8.68, parents = 0, gradient = 0.0,
  adiff value = 0.0, parents = 0, gradient = 0.0,
  adiff value = 0.35, parents = 0, gradient = 0.0],
 [adiff value = 1.52081, parents = 0, gradient = 0.0,
  adiff value = 13.78, parents = 0, gradient = 0.0,
  adiff value = 2.28, parents = 0, gradient = 0.0,
  adiff value = 1.43, parents = 0, gradient = 0.0,
  adiff value = 71.99, parents = 0, gradient = 0.0,
  adiff value = 0.49, parents = 0, gradient = 0.0,
  adiff value = 9.85, parents = 0, gradient = 0.0,
  adiff value = 0.0, parents = 0, gradient = 0.0,
  adiff value = 0.17, parents = 0, gradient = 0.0],
 [adiff value = 1.51708, parents = 0, gradient = 0.0,
  adiff value = 13

In [ ]:

output=testingthing.Network[4].next_layer_input
np.argwhere(output==np.amax(output,axis=1).reshape(output.shape[0],1))[:,1]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
np.ravel(matrix_data(train)[1])

array([2, 2, 2, 2, 1, 2, 2, 5, 2, 7, 1, 2, 7, 1, 1, 1, 7, 2, 2, 1, 2, 7,
       7, 6, 3, 2, 6, 2, 6, 7, 1, 2, 6, 2, 7, 1, 2, 1, 1, 2, 7, 2, 1, 1,
       2, 1, 3, 2, 2, 2, 2, 1, 7, 7, 1, 5, 1, 1, 1, 1, 1, 3, 1, 2, 3, 7,
       2, 2, 2, 1, 3, 6, 5, 2, 1, 6, 2, 2, 7, 2, 1, 1, 3, 5, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 3, 1, 3, 7, 2, 5, 1,
       2, 1, 1, 1, 7, 3, 1, 3, 5, 2, 2, 5, 7, 2, 1, 7, 2, 3, 7, 7, 2, 1,
       7, 1, 1, 2, 2, 2, 5, 2, 5, 1, 2, 1, 1, 2, 7, 1, 1])

In [ ]:
np.amax(output,axis=1)

array([0.35629773, 0.35627203, 0.35609579, 0.35611337, 0.35607858,
       0.35607057, 0.35623628, 0.35614003, 0.35602834, 0.35630093,
       0.35622909, 0.35619161, 0.35633964, 0.35596704, 0.35608197,
       0.35607274, 0.3563268 , 0.35606341, 0.35600757, 0.35610186,
       0.3560863 , 0.35622709, 0.35617913, 0.35606704, 0.3560053 ,
       0.35619109, 0.35600853, 0.35608212, 0.35611777, 0.35619342,
       0.35626186, 0.35602851, 0.35602522, 0.3560683 , 0.35627142,
       0.3560628 , 0.35607133, 0.35621684, 0.3560652 , 0.35621919,
       0.3561796 , 0.35607542, 0.35607465, 0.35603079, 0.3562459 ,
       0.3562639 , 0.35599144, 0.35629977, 0.35598286, 0.35599383,
       0.35609562, 0.35625156, 0.35622902, 0.3562803 , 0.35618193,
       0.35615526, 0.35608186, 0.35620765, 0.35617608, 0.35607873,
       0.35618327, 0.3563083 , 0.3560579 , 0.35627167, 0.35599832,
       0.35621373, 0.35611709, 0.35612123, 0.35623599, 0.35622148,
       0.35601378, 0.35605507, 0.35629814, 0.35611959, 0.35606

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58866f15-1cff-4fef-9525-5c3070562370' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>